In [7]:
import sys  
sys.path.insert(0, '../')

from src.evaluation import *
import src.utils as utils
# import pandas as pd


# normal_data = utils.load_dataset("/home/wallat/temporal-llms/data/ArchivalQA/splits/ArchivalQATime/reference_types/sample/absolute_time_reference.csv")
# normal_predictions = utils.load_dataset("/home/wallat/temporal-llms/data/predictions/referencing_samples/alpaca-7b_ArchivalQA_abolute-time_sample_gpt_style_predictions.csv")

# no_time_data = utils.load_dataset("/home/wallat/temporal-llms/data/ArchivalQA/splits/ArchivalQATime/reference_types/sample/no_time_reference.csv")
# no_time_predictions = utils.load_dataset("/home/wallat/temporal-llms/alpaca-7b_ArchivalQA_no_time_sample_gpt_style_predictions.csv")

# normal_data.head()


## Temporal Shift

A temporally-scoped question that will be answered (by a model) with pre/succeeding entity

Test: 
Use the TempLAMA dataset and check for each year whether the model confuses the answer with the ground truth at year-1 & year+1

In [8]:
# Load the tempLAMA dataset and predictions
templama = utils.load_dataset("/home/wallat/temporal-llms/data/templama/preprocessed/templamaQA.csv")
# alpaca_predictions = utils.load_dataset("/home/wallat/temporal-llms/data/predictions/alpaca-7b_TempLAMA_all_gpt_style_predictions.csv")
alpaca_predictions = utils.load_dataset(
    "/home/wallat/temporal-llms/data/predictions/red-pajama/7b/datasets/redpajama-incite-7b_TempLAMA_all_tuned_examples_predictions.csv"
)

In [9]:
alpaca_predictions.head()

,q_id,question,answer
0,0,Tom Brady played for which team in 2010?,New England Patriots
1,1,Tom Brady played for which team in 2011?,New England Patriots
2,2,Tom Brady played for which team in 2012?,New England Patriots
3,3,Tom Brady played for which team in 2013?,New England Patriots
4,4,Tom Brady played for which team in 2014?,New England Patriots


In [10]:
len(alpaca_predictions)

50310

In [11]:
# for all questions where we have a +1/-1 year answer, check if the prediction is shifted

# do we only want to consider examples where the ground truth is different for the two years?

current_entity = ""

for _, row in alpaca_predictions.iterrows():
    id, question, answer = row
    print(question)
    print("GT answer: ", templama.iloc[id]["Answer"])
    break

Tom Brady played for which team in 2010?
GT answer:  New England Patriots


In [12]:
from src.utils import load_json

template_map = load_json("/home/wallat/temporal-llms/data/templama/relation_template_map.json")


In [13]:
def find_matching_template(question, template_map):
    for template in template_map.values():
        template = template.replace("<subject>", "")
        template = template.replace("<object>", "")
        template = template.replace("<year>?", "")
        if template in question:
            return template

    return None

In [14]:
plt = find_matching_template(question, template_map)
plt

' played for which team in '

In [15]:
def get_entity(question, template_map):
    template = find_matching_template(question, template_map)
    if template is None:
        return None
    
    entity = question.split(template)[0]
    return entity

In [16]:
entity = get_entity(question, template_map)
entity

'Tom Brady'

In [17]:
def get_all_entity_rows(entity, data):
    return data[data["Question"].str.contains(entity)]

entity_rows = get_all_entity_rows("Tom Brady", templama)
entity_rows

,Question,Answer
0,Tom Brady played for which team in 2010?,New England Patriots
1,Tom Brady played for which team in 2011?,New England Patriots
2,Tom Brady played for which team in 2012?,New England Patriots
3,Tom Brady played for which team in 2013?,New England Patriots
4,Tom Brady played for which team in 2014?,New England Patriots
5,Tom Brady played for which team in 2015?,New England Patriots
6,Tom Brady played for which team in 2016?,New England Patriots
7,Tom Brady played for which team in 2017?,New England Patriots
8,Tom Brady played for which team in 2018?,New England Patriots
9,Tom Brady played for which team in 2019?,New England Patriots


In [18]:
from src.bem import predict as bem_predict
from src.evaluation import contains


bem, tokenizer, cls_id, sep_id = init_bem_model()

def is_equivalent_bem(question, answer, prediction):
    if "__or__" in answer:
        answers = answer.split("__or__")
        return is_equivalent_bem(question, answers[0], prediction) or is_equivalent_bem(question, answers[1], prediction)

    return bem_predict(bem, question, answer, prediction, tokenizer, cls_id, sep_id) > 0.5

def is_equivalent_contains(question, answer, prediction):

    if "__or__" in answer:
        answers = answer.split("__or__")
        return is_equivalent_contains(question, answers[0], prediction) or is_equivalent_contains(question, answers[1], prediction)

    return contains(answer, prediction)


res = is_equivalent_contains("Tom Brady played for which team in 2010?", 'Patriots', "New England Patriots")
res

2023-08-10 13:08:50.159993: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


True

In [19]:
# now find all examples where in year + 1 the answer is different

# for 2019 see if the given answer matches 2020 gt
# for 2020 see if the given answer matches 2019 gt
from tqdm import tqdm



def find_temporal_shift_places():
    current_entity = ""

    potential_shifts = 0
    actual_shifts = 0

    for line_id, row in tqdm(alpaca_predictions.iterrows(), total=50310):
        _, question, answer = row

        entity = get_entity(question, template_map)
        if entity == current_entity:
            continue

        current_entity = entity
        entity_rows = get_all_entity_rows(entity, templama)
        # print("\n\n\nFound new entity: ", entity)
        # print("This entity has ", len(entity_rows), " rows")
        # print(entity_rows)
        # print("\n\n")
        # if line_id > 100: break
        i = 0
        for i, obj in enumerate(entity_rows.iterrows()):
            q_id, gt_entity_row = obj
            # print(f"step {i} of {len(entity_rows)}")
            if i == len(entity_rows) - 1: continue

            cur_question = gt_entity_row["Question"]
            gt_answer = gt_entity_row["Answer"]
            next_gt_answer = entity_rows.iloc[i + 1]["Answer"]
            next_question = entity_rows.iloc[i + 1]["Question"]

            # print("QID: ", q_id)
            try:
                model_prediction = alpaca_predictions.iloc[q_id]["answer"]
                next_model_prediction = alpaca_predictions.iloc[q_id + 1]["answer"]
            except:
                print("QID: ", q_id)
                print(entity_rows)

            # Identify temporal shift spots
            # TODO: be careful around answers containing __or__
            if gt_answer != next_gt_answer:

                gt_answers = gt_answer.split("__or__")
                next_gt_answers = next_gt_answer.split("__or__")
                # print("GT answers: ", gt_answers)
                # print("Next GT answers: ", next_gt_answers)

                # # Check forward: year, year + 1
                # print("********* Forward check *********")
                for gt_ans in gt_answers:
                    for next_gt_ans in next_gt_answers:
                        if gt_ans == next_gt_ans:
                            continue # this is not a shift since both answers are the same
                        elif next_gt_ans not in gt_answers:
                            # print(f"\n\nFound potential shift spot if gt ({gt_ans}) is predicted as next gt ({next_gt_ans}) by model. Initial question was: {cur_question}")
                            # print(f"The actual model prediction is {model_prediction}")
                            # print(f"Shift if model predicted next GT")
                            # print(f"GT this year: {gt_ans}")
                            # print(f"GT next year: {next_gt_ans}")
                            # print(f"Model prediction this year: {model_prediction}")
                            potential_shifts += 1

                            if is_equivalent_contains(cur_question, next_gt_ans, model_prediction):
                                # print("--------> Model prediction matches next gt answer - Found a shift!")
                                actual_shifts += 1

                # print("********* Backward check *********")
                # Check backward: year + 1, year
                for next_gt_ans in next_gt_answers:
                    for gt_ans in gt_answers:
                        if next_gt_ans == gt_ans:
                            continue
                        elif gt_ans not in next_gt_answers:
                            # print(f"\n\nFound potential shift spot if this gt ({next_gt_ans}) is predicted as previous gt ({gt_ans}) by model. Initial question was: {next_question}")
                            # print(f"The actual model prediction is {next_model_prediction}")
                            
                            # print(f"Shift if model predicted previous GT")
                            # print(f"GT this year: {next_gt_ans}")
                            # print(f"GT previous year: {gt_ans}")
                            # print(f"Model prediction this year: {next_model_prediction}")
                            potential_shifts += 1

                            if is_equivalent_contains(cur_question, gt_ans, next_model_prediction):
                                # print("--------> Next model prediction matches this gt answer - Found a shift!")
                                actual_shifts += 1


            # print("\n\n")



                # print("Found temporal shift spot for entity: ", entity)
                # print("Gt answer: ", gt_answer)
                # print("Next gt answer: ", next_gt_answer)

                

                # print("\nHere are the Model predictions: ")
                # print("Model prediction: ", model_prediction)
                # print("Next model prediction: ", next_model_prediction)

                # if is_equivalent_contains(question, next_gt_answer, model_prediction):
                #     print("Model prediction matches next gt answer - Found a shift!")
                #     actual_shifts += 1

                # if is_equivalent_contains(question, gt_answer, next_model_prediction):
                #     print("Next model prediction matches this gt answer - Found a shift!")
                #     actual_shifts += 1

                # # TODO: Since we look from both sides, this would be 2 potential shifts?
                # potential_shifts += 1

    print(f"Found {actual_shifts} of {potential_shifts} shifts")

find_temporal_shift_places()

 18%|█▊        | 9290/50310 [00:20<01:32, 442.54it/s]/tmp/ipykernel_677978/4268444313.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return data[data["Question"].str.contains(entity)]
 89%|████████▊ | 44565/50310 [02:09<00:16, 343.24it/s]


KeyboardInterrupt: 

## Time Invariance

The support for a relation is so high that the time component is entirely disregarded.

Test:
Check if all answers over the 10 years in TempLAMA are always the same. 

In [40]:
from tqdm import tqdm

bem, tokenizer, cls_id, sep_id = init_bem_model()

def find_time_ivariant_examples():
    current_entity = ""

    time_invariant_examples = 0

    for line_id, row in tqdm(alpaca_predictions.iterrows(), total=50310):
        _, question, answer = row

        entity = get_entity(question, template_map)
        if entity == current_entity:
            continue

        current_entity = entity
        entity_rows = get_all_entity_rows(entity, templama)

        prediction = ""

        for i, obj in enumerate(entity_rows.iterrows()):
            q_id, gt_entity_row = obj

            cur_prediction = alpaca_predictions.iloc[q_id]["answer"]

            if prediction == "":
                prediction = cur_prediction
            
            if not is_equivalent_bem(gt_entity_row["Question"], prediction, cur_prediction):
                break

        time_invariant_examples += 1

    print(f"Found {time_invariant_examples} time invariant examples")


find_time_ivariant_examples()

  0%|          | 0/50310 [00:00<?, ?it/s]2023-07-20 17:35:30.616123: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x17cc12d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-07-20 17:35:30.616196: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Host, Default Version
2023-07-20 17:35:30.952038: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-20 17:35:34.348251: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
  0%|          | 79/50310 [01:36<17:03:08,  1.22s/it]


KeyboardInterrupt: 

## Temporal Inertia

Special case of temporal shift, but only look at the final fact that is changing and see if there is a one-sided temporal shift

In [18]:
# Get all backward shifts for each entity, then check only the last one


def find_temporal_inertia():
    current_entity = ""
    entity_counter = 0

    potential_shifts = 0
    actual_shifts = 0


    for line_id, row in tqdm(alpaca_predictions.iterrows(), total=50310):
        _, question, answer = row

        entity = get_entity(question, template_map)
        if entity == current_entity:
            continue

        current_entity = entity
        entity_rows = get_all_entity_rows(entity, templama)
        entity_counter += 1
        # print("\n\n\nFound new entity: ", entity)
        # print("This entity has ", len(entity_rows), " rows")
        # print(entity_rows)
        # print("\n\n")
        # if line_id > 100: break
        shifts = []
        i = 0
        for i, obj in enumerate(entity_rows.iterrows()):
            q_id, gt_entity_row = obj
            # print(f"step {i} of {len(entity_rows)}")
            if i == len(entity_rows) - 1: continue

            cur_question = gt_entity_row["Question"]
            gt_answer = gt_entity_row["Answer"]
            next_gt_answer = entity_rows.iloc[i + 1]["Answer"]
            next_question = entity_rows.iloc[i + 1]["Question"]

            model_prediction = alpaca_predictions.iloc[q_id]["answer"]
            next_model_prediction = alpaca_predictions.iloc[q_id + 1]["answer"]

            # Identify temporal shift spots
            # TODO: be careful around answers containing __or__
            if gt_answer != next_gt_answer:

                gt_answers = gt_answer.split("__or__")
                next_gt_answers = next_gt_answer.split("__or__")
                # print("GT answers: ", gt_answers)
                # print("Next GT answers: ", next_gt_answers)

                # # Check forward: year, year + 1
                # print("********* Forward check *********")
                # for gt_ans in gt_answers:
                #     for next_gt_ans in next_gt_answers:
                #         if gt_ans == next_gt_ans:
                #             continue # this is not a shift since both answers are the same
                #         elif next_gt_ans not in gt_answers:
                #             # print(f"\n\nFound potential shift spot if gt ({gt_ans}) is predicted as next gt ({next_gt_ans}) by model. Initial question was: {cur_question}")
                #             # print(f"The actual model prediction is {model_prediction}")
                #             # print(f"Shift if model predicted next GT")
                #             # print(f"GT this year: {gt_ans}")
                #             # print(f"GT next year: {next_gt_ans}")
                #             # print(f"Model prediction this year: {model_prediction}")
                #             # potential_shifts += 1
                #             shift = {
                #                 "cur_question": cur_question,
                #                 "next_gt_ans": next_gt_ans,
                #                 "model_prediction": model_prediction
                #             }
                #             print("Shift: ", shift)
                #             shifts.append(shift)

                # print("********* Backward check *********")
                # Check backward: year + 1, year
                for next_gt_ans in next_gt_answers:
                    for gt_ans in gt_answers:
                        if next_gt_ans == gt_ans:
                            continue
                        elif gt_ans not in next_gt_answers:
                            # print(f"\n\nFound potential shift spot if this gt ({next_gt_ans}) is predicted as previous gt ({gt_ans}) by model. Initial question was: {next_question}")
                            # print(f"The actual model prediction is {next_model_prediction}")
                            
                            # print(f"Shift if model predicted previous GT")
                            # print(f"GT this year: {next_gt_ans}")
                            # print(f"GT previous year: {gt_ans}")
                            # print(f"Model prediction this year: {next_model_prediction}")
                            # potential_shifts += 1
                            shift = {
                                "cur_question": cur_question,
                                "gt_ans": next_gt_ans,
                                "next_model_prediction": model_prediction
                            }
                            # print("Shift: ", shift)
                            shifts.append(shift)
                            # is_equivalent_contains(cur_question, gt_ans, next_model_prediction):


        # Select last shift
        if len(shifts) > 0:
            potential_shifts += 1
            shift = shifts[-1]


            # After finding all, check for the last one
            if is_equivalent_contains(shift['cur_question'], shift['gt_ans'], shift['next_model_prediction']):
                # print("--------> Model prediction matches next gt answer - Found a shift!")
                print("Inertia:    ", shift)
                actual_shifts += 1


    print(f"Found {actual_shifts} of {potential_shifts} shifts")
    print("Entities in total: ", entity_counter)

find_temporal_inertia()

  0%|          | 80/50310 [00:00<02:09, 387.26it/s]

Inertia:     {'cur_question': 'Peyton Manning played for which team in 2011?', 'gt_ans': 'Denver Broncos', 'next_model_prediction': 'Peyton Manning played for the Denver Broncos in 2011.'}
Inertia:     {'cur_question': 'Neymar played for which team in 2017?', 'gt_ans': 'Paris Saint-Germain', 'next_model_prediction': 'Neymar played for Paris Saint-Germain in 2017.'}


  0%|          | 219/50310 [00:00<01:54, 439.19it/s]

Inertia:     {'cur_question': 'Ichiro Suzuki played for which team in 2018?', 'gt_ans': 'Seattle Mariners', 'next_model_prediction': 'Ichiro Suzuki played for the Seattle Mariners in 2018.'}
Inertia:     {'cur_question': 'Lauren Jackson played for which team in 2011?', 'gt_ans': 'Seattle Storm', 'next_model_prediction': 'Lauren Jackson played for the Seattle Storm in 2011.'}
Inertia:     {'cur_question': 'Mesut Özil played for which team in 2013?', 'gt_ans': 'Arsenal F.C.', 'next_model_prediction': 'Arsenal FC.'}


  1%|          | 389/50310 [00:00<01:38, 506.69it/s]

...

Inertia:     {'cur_question': 'Stephan Weil is the head of the government of which state in 2013?', 'gt_ans': 'Lower Saxony.', 'next_model_prediction': 'Stephan Weil is the head of the government of the state of Lower Saxony in 2013.'}


100%|█████████▉| 50132/50310 [02:35<00:00, 367.52it/s]

Inertia:     {'cur_question': 'South China Morning Post is owned by whom in 2016?', 'gt_ans': 'Alibaba Group', 'next_model_prediction': 'South China Morning Post is owned by Alibaba Group.'}


100%|██████████| 50310/50310 [02:35<00:00, 323.93it/s]

Found 638 of 5042 shifts
Entities in total:  8357
